## Importación de librerías y declaración de funciones

In [1]:
import diseño_registro
from lector_archivo import ArchivoPresentacion

def dbl_con_ceros(num, cant):
    return "{:.2f}".format(num).replace('.','').zfill(cant)

## Apertura del TXT

In [2]:
txt = ArchivoPresentacion(r"A11_MESCONVACSINTOPE_A1_PRUEBA_20221031172633.txt")

## Modificación de datos dentro del TXT

### [Ejemplo] Modificación del código de liquidación para ciertos CUILs:

In [10]:
cuils_modificar = """
20276914155
20299391583
20264959013
20369619706
20320995680
20237969341
20355259839
20279576463
20254068056
23315494699
23205421769
20289467220
20181290405
"""

# Respetar el ancho del string
codigo_anterior = 'VAC9      '
codigo_nuevo = '/109A     '

In [11]:
cuils_modificar = cuils_modificar.strip().split('\n')

for emp in txt.empleados:
    if emp in cuils_modificar: # Si es para todos, esta comprobación no se haría.
        for concepto in emp.conceptos:
            if concepto['codigo_concepto_liquidado'] == codigo_anterior:
                concepto['codigo_concepto_liquidado'] = codigo_nuevo
                concepto['cantidad'] = '00000'
                concepto['unidades'] = ' '
            

In [6]:
cuils_modificar = cuils_modificar.strip().split('\n')

for emp in txt.empleados:
    if emp in cuils_modificar:
        emp.registro2['dias_prop_tope'] = '030'

### [Ejemplo] Verificación de que la remuneracion 10 no sea menor al MNI:

In [5]:
MINIMO_NO_IMPONIBLE = 14601.14

In [6]:
for emp in txt.empleados:
    base10 = int(emp.registro4['base_imponible10'])/100.0
    
    if base10 < MINIMO_NO_IMPONIBLE:

        print(f"CUIL: {emp}")
        print("La base_imponible10 es menor al mínimo no imponible.")
        
        base8 = int(emp.registro4['base_imponible8'])/100.0
        detraccion = max(0,round(base10 - base8,2))
        
        emp.registro4['base_imponible10'] = dbl_con_ceros(MINIMO_NO_IMPONIBLE, diseño_registro.DISEÑO4['base_imponible10'])
        emp.registro4['importe_detraer'] = dbl_con_ceros(detraccion, diseño_registro.DISEÑO4['importe_detraer'])
        
        print("Se actualizaron los siguientes valores:")
        print(f" base_imponible10: {emp.registro4['base_imponible10']}")
        print(f" importe_detraer: {emp.registro4['importe_detraer']}")


CUIL: 20367518503
La base_imponible10 es menor al mínimo no imponible.
Se actualizaron los siguientes valores:
 base_imponible10: 000000001460114
 importe_detraer: 000000000000000


### [Ejemplo] Incorporación del SAC a determinados CUILs

In [7]:
# Pegar a continuación:
#  CUIT Cnt.Dias Importe
info = """
27257004290	120	100,5
20290290393	120	100,5
"""


info = info.strip().split('\n')

cuils_agregar = {}
for i in info:
    cortado = i.split('\t')
    cuils_agregar[cortado[0]] = {'dias': cortado[1], 'importe': float(cortado[2].replace(',','.'))}

cuils_agregar

{'27257004290': {'dias': '120', 'importe': 100.5},
 '20290290393': {'dias': '120', 'importe': 100.5}}

In [8]:
for emp in txt.empleados:
    if emp in list(cuils_agregar.keys()):

        dias = str(cuils_agregar[emp.cuit]['dias']).zfill(5)
        importe = dbl_con_ceros(cuils_agregar[emp.cuit]['importe'], 15)

        emp.conceptos.append({
            'registro':                     '03',
            'cuil':                         emp.cuit,
            'codigo_concepto_liquidado':    '/S13      ',
            'cantidad':                     dias,
            'unidades':                     'D',
            'importe':                      importe,
            'debcre':                       'C',
            'periodo_retro':                '      '
        })

## [Ejemplo] Asignación de seguro de vida a Pasantes

In [3]:
txt = ArchivoPresentacion(r"S:\Rrhh\Liq-Haberes\Año 2022\10 - Octubre 2022\LSD\Mensual\A10_MESPASANTES_A24.txt")

In [4]:
for emp in txt.empleados:
    if emp.registro4['modalidad_contratacion'] == '027':
        emp.registro4['marca_scvo'] = '1'

## [Ejemplo] Eliminar registros de determinados CUILs

In [9]:
cuil_eliminar = """
27408691201
20395468457

"""
cuil_eliminar = cuil_eliminar.strip().split('\n')

for emp in txt.empleados:
    if emp in cuil_eliminar:
        txt.empleados.remove(emp)

In [14]:
import math
TOPE_MENSUAL = 474_530.27
TOPE_DIARIO = TOPE_MENSUAL / 30

print('CUIL', 'Rem1', 'Rem4', 'Rem5', 'dias_orig', 'dias_modif')
for emp in txt.empleados:

    # Remuneraciones con tope: 1, 4 y 5
    rem1 = int(emp.registro4['base_imponible1'])/100.0
    rem4 = int(emp.registro4['base_imponible4'])/100.0
    rem5 = int(emp.registro4['base_imponible5'])/100.0
    dias = int(emp.registro2['dias_prop_tope'])

    tope_aplicable = TOPE_DIARIO * dias

    control = max(rem1, rem4, rem5) > tope_aplicable

    if control:  
        
        dias_suficiente = math.ceil(max(rem1, rem4, rem5) / TOPE_DIARIO)
        dias_suficiente = str(dias_suficiente).zfill(3)

        emp.registro2['dias_prop_tope'] = dias_suficiente
        print(emp, rem1, rem4, rem5, dias, dias_suficiente)

CUIL Rem1 Rem4 Rem5 dias_orig dias_modif
20296183084 476824.87 476824.87 476824.87 29 031
20270142649 490432.07 490432.07 490432.07 23 032
20251758507 549738.18 549738.18 549738.18 29 035
20301224819 471709.47 471709.47 471709.47 23 030
20275428834 474530.27 474530.27 474530.27 29 030
20133704176 431249.24 431249.24 431249.24 16 028
20213912403 453787.5 453787.5 453787.5 16 029
20303089366 445600.67 445600.67 445600.67 28 029
20160822164 482175.88 482175.88 482175.88 23 031
20292532378 474854.12 474854.12 474854.12 24 031
20309002645 460562.31 460562.31 460562.31 29 030
20246150002 390092.68 390092.68 390092.68 21 025
20292551208 434555.91 434555.91 434555.91 23 028
20337012257 389236.17 389236.17 389236.17 22 025
20279200269 488055.16 488055.16 488055.16 24 031
20264844836 505340.12 505340.12 505340.12 16 032
20227721368 463288.61 463288.61 463288.61 23 030
20184302021 315721.96 315721.96 315721.96 12 020
20223750894 377879.48 377879.48 377879.48 23 024
23162377809 414768.93 414768.93

## Guardado del archivo modificado

In [10]:
#txt.guardar(inplace=True)
txt.guardar()

Se guardo el txt correctamente:
A10_MESPASANTES_A24_20221101102331.txt
